In [1]:
import os
import fastf1
import pandas as pd
from fastf1 import utils
from tqdm import tqdm

C:\Users\lenovo\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# CREATING A CACHE DIRECTORY

cache_path = 'f1_cache'
os.makedirs(cache_path, exist_ok=True)

In [3]:
# ENABLING THE CACHE

fastf1.Cache.enable_cache('f1_cache')

In [4]:
# SETTINGS

year = 2024
event = 'Silverstone'
session_type = 'R'  # Main Race
telemetry_driver = 'VER'

In [5]:
# LOADING THE SESSION

session = fastf1.get_session(year, event, session_type)
session.load()

core           INFO 	Loading data for British Grand Prix - Race [v3.3.9]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
logger      WARNING 	Failed to add first lap time from Ergast!
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
c

In [6]:
# GET ALL LAPS (All Drivers)

laps = session.laps.reset_index(drop=True)
laps['DriverLapKey'] = laps['Driver'] + '_' + laps['LapNumber'].astype(str)

In [7]:
# CLEANING THE TIME COLUMNS

time_cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'PitInTime', 'PitOutTime']
for col in time_cols:
    laps[col + '_s'] = laps[col].dt.total_seconds()

In [8]:
# LAP TIME STRING FOR TOOLTIP 

laps['LapTime_str'] = laps['LapTime'].astype(str).str.extract(r'(\d{2}:\d{2}\.\d{3})')[0]

In [9]:
# FINAL LAP DATASET

lap_data_cleaned = laps[[
    'Driver', 'Team', 'LapNumber', 'DriverLapKey', 'Compound', 'Stint',
    'LapTime_str', 'LapTime_s',
    'Sector1Time_s', 'Sector2Time_s', 'Sector3Time_s',
    'PitInTime_s', 'PitOutTime_s',
    'IsPersonalBest', 'IsAccurate'
]]

In [12]:
# TELEMETRY: VERSTAPPEN

ver_laps = laps[laps['Driver'] == telemetry_driver]
telemetry_list = []

In [13]:
for _, lap in ver_laps.iterrows():
    try:
        tel = lap.get_telemetry().copy()
        tel['Driver'] = lap['Driver']
        tel['LapNumber'] = lap['LapNumber']
        tel['DriverLapKey'] = lap['DriverLapKey']
        tel['SessionTime_s'] = tel['SessionTime'].dt.total_seconds()
        tel['Time_s'] = tel['Time'].dt.total_seconds()
        tel['DistanceRounded'] = tel['Distance'].round(1)

        # Time strings
        tel['SessionTime_str'] = tel['SessionTime'].apply(
            lambda td: f"{int(td.total_seconds() // 60):02}:{int(td.total_seconds() % 60):02}.{int((td.total_seconds() % 1) * 1000):03}"
        )
        tel['LapTime_str'] = tel['Time'].apply(
            lambda td: f"{int(td.total_seconds() // 60):02}:{int(td.total_seconds() % 60):02}.{int((td.total_seconds() % 1) * 1000):03}"
        )

        telemetry_list.append(tel)
    except Exception as e:
        print(f"⚠️ Skipped lap {lap['LapNumber']} due to telemetry error: {e}")

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core      

In [14]:
# COMBINING THE TELEMETRY

ver_telemetry = pd.concat(telemetry_list, ignore_index=True)

In [15]:
# ROUNDING UP NUMERIC FIELDS

ver_telemetry = ver_telemetry.round({
    'Speed': 1, 'Throttle': 2, 'Brake': 2, 'RPM': 0,
    'DistanceToDriverAhead': 2, 'RelativeDistance': 2
})

In [16]:
# FINAL TELEMETRY DATASET

telemetry_cleaned = ver_telemetry[[
    'Driver', 'LapNumber', 'DriverLapKey',
    'Distance', 'DistanceRounded',
    'Speed', 'Throttle', 'Brake', 'RPM', 'nGear', 'DRS',
    'RelativeDistance', 'DistanceToDriverAhead',
    'SessionTime_s', 'SessionTime_str', 'Time_s', 'LapTime_str'
]]

In [42]:
# EXPORT TO CSV

telemetry_cleaned.to_csv('telemetry_verstappen_silverstone_race.csv', index=False)